# Test NSEG dataset

In [74]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Masking, Bidirectional
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [69]:
df = pd.read_csv(".\EEG_EMOTION\dataset_de_LDS_SEEDIV.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [70]:
df[df.isnull()==True] = -10

In [71]:
def preprocessing_for_LSTM(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = to_categorical(y_train)
    X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Subject dependent

In [72]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [ ]:
accuracy_test = []
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    model = Sequential()
    model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
    model.add(Bidirectional(GRU(units=128, input_shape=(X_trainSD.shape[1], 1))))
    model.add(Dense(units=4, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=15, batch_size=64)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

Epoch 1/15
4/4 [==============================] - 262s 63s/step - loss: 4.9505 - accuracy: 0.2458
Epoch 2/15
4/4 [==============================] - 258s 64s/step - loss: 4.8846 - accuracy: 0.2292
Epoch 3/15
4/4 [==============================] - 260s 66s/step - loss: 4.8520 - accuracy: 0.2500
Epoch 4/15
4/4 [==============================] - 308s 78s/step - loss: 4.8411 - accuracy: 0.3125
Epoch 5/15
4/4 [==============================] - 367s 92s/step - loss: 4.8373 - accuracy: 0.3125
Epoch 6/15
3/4 [=====================>........] - ETA: 1:25 - loss: 4.8400 - accuracy: 0.3125

In [ ]:
print("ACCURACY: ", np.array(accuracy_test).mean())

# Subject-Biased Experiment


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)
X_train, X_test, y_train, y_test = preprocessing_for_LSTM(X_train, X_test, y_train, y_test)

In [ ]:
del model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=4, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

# Subject Indipendent

In [ ]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

# TO DO: da implementare kfold
https://stackoverflow.com/questions/48085182/cross-validation-in-keras


"""
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def buildmodel():
    model= Sequential([
        Dense(10, activation="relu"),
        Dense(5, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return(model)

estimator= KerasRegressor(build_fn=buildmodel, epochs=100, batch_size=10, verbose=0)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, x, y, cv=kfold, n_jobs=2)  # 2 cpus
results.mean()  # Mean MSE
"""

In [ ]:
scores_test = []
logo = LeaveOneGroupOut()
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    X_SI = np.array(X_SI).reshape((X_SI.shape[0], X_SI.shape[1], 1))
    y_SI = to_categorical(y_SI)
    del model
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(int(X_SI.shape[1])-15, 1)))
    model.add(Dense(units=4, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    scores_gbc = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())